This notebook:
1. Loads and cleans the raw data
2. Prepares the data for the Anserini retriever
3. Pre-processes and tokenizes the raw cleaned data
4. Creates vocabulary from the corpus

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


In [ ]:
from statistics import mean 

from src.process_data import *

# **Load data**

In [ ]:
data_path = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
# Document id and Answer text
collection = load_answers_to_df(data_path+"FiQA_train_doc_final.tsv")
# Question id and Question text
queries = load_questions_to_df(data_path+"FiQA_train_question_final.tsv")
# Question id and Answer id pair
qid_docid = load_qid_docid_to_df(data_path+"FiQA_train_question_doc_final.tsv")

In [ ]:
print("Document id and Answer text")
collection.head(5)

Document id and Answer text


,docid,doc
0,3,I'm not saying I don't like the idea of on-the...
1,31,So nothing preventing false ratings besides ad...
2,56,You can never use a health FSA for individual ...
3,59,Samsung created the LCD and other flat screen ...
4,63,Here are the SEC requirements: The federal sec...


In [ ]:
print("Example answer: \n")
print(collection.iloc[0]['doc'])

Example answer: 

I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.


In [ ]:
print("Question id and Question text")
queries.head(5)

Question id and Question text


,qid,question
0,0,What is considered a business expense on a bus...
1,1,Claiming business expenses for a business with...
2,2,Transferring money from One business checking ...
3,3,Having a separate bank account for business/in...
4,4,Business Expense - Car Insurance Deductible Fo...


In [ ]:
print("Example Question: \n")
print(queries.iloc[0]['question'])

Example Question: 

What is considered a business expense on a business trip?


In [ ]:
print("Question id and Answer id pair")
qid_docid.head(5)

Question id and Answer id pair


,qid,docid
0,0,18850
1,1,14255
2,2,308938
3,3,296717
4,3,100764


In [ ]:
qid_rel = label_to_dict(qid_docid)

In [ ]:
# Number of relevant passages for each query
num_rel = [len(v) for v in qid_rel.values()]

avg_num_rel = mean(num_rel)
max_num_rel = max(num_rel)
min_num_rel = min(num_rel)

print("Average number of relevant passages for each query: {}\n".format(round(avg_num_rel)))
print("Max number of relevant passages for each query: {}\n".format(max_num_rel))
print("Min number of relevant passages for each query: {}\n".format(min_num_rel))

Average number of relevant passages for each query: 3

Max number of relevant passages for each query: 23

Min number of relevant passages for each query: 1



In [ ]:
print("Example QA pair: \n")
print("Question: {}".format(queries.at[3, 'question']))

x = collection[collection['docid']==296717]

print()
print("Answer: {}".format(x.at[28732, 'doc']))

Example QA pair: 

Question: Having a separate bank account for business/investing, but not a “business account?”

Answer: Having a separate checking account for the business makes sense. It simplifies documenting your income/expenses. You can "explain" every dollar entering and exiting the account without having to remember that some of them were for non-business items. My credit union allowed me to have a 2nd checking account and allowed me to put whatever I wanted as the name on the check. I think this looked a little better than having my name on the check. I don't see the need for a separate checking account for investing. The money can be kept in a separate savings account that has no fees, and can even earn a little interest. Unless you are doing a lot of investment transactions a month this has worked for me. I fund IRAs and 529 plans this way. We get paychecks 4-5 times a month, but send money to each of the funds once a month. You will need a business account if the number of

In [ ]:
print("Number of answers: {}".format(len(collection)))
print("Number of questions: {}".format(len(queries)))
print("Number of QA pairs: {}".format(len(qid_docid)))

Number of answers: 57638
Number of questions: 6648
Number of QA pairs: 17110


# **Clean data**

In [ ]:
# Cleaning data
empty_docs, empty_id = get_empty_docs(collection)
# Remove empty answers from collection of answers
collection_cleaned = collection.drop(empty_id)
# Remove empty answers from qa pairs
qid_docid = qid_docid[~qid_docid['docid'].isin(empty_docs)]

print("Number of answers after cleaning: {}".format(len(collection_cleaned)))
print("Number of QA pairs after cleaning: {}".format(len(qid_docid)))

Number of answers after cleaning: 57600
Number of QA pairs after cleaning: 17072


# **Prepare data for Anserini**

In [ ]:
# Write collection df to file
save_tsv("retriever/collection_cleaned.tsv", collection_cleaned)

# Convert collection df to JSON file for Anserini's document indexer
collection_to_json("retriever/collection_json/docs.json", "retriever/collection_cleaned.tsv")

# **Process Data**

In [ ]:
processed_answers = process_answers(collection_cleaned)
processed_questions = process_questions(queries)

In [ ]:
print("Processed and tokenized questions")
processed_questions.head(5)

Processed and tokenized questions


,qid,question,q_processed,tokenized_q,q_len
0,0,What is considered a business expense on a bus...,what is considered a business expense on a bus...,"[what, is, considered, a, business, expense, o...",10
1,1,Claiming business expenses for a business with...,claiming business expenses for a business with...,"[claiming, business, expenses, for, a, busines...",9
2,2,Transferring money from One business checking ...,transferring money from one business checking ...,"[transferring, money, from, one, business, che...",10
3,3,Having a separate bank account for business/in...,having a separate bank account for business in...,"[having, a, separate, bank, account, for, busi...",13
4,4,Business Expense - Car Insurance Deductible Fo...,business expense car insurance deductible fo...,"[business, expense, car, insurance, deductible...",13


In [ ]:
print("Processed and tokenized answers")
processed_answers.head(5)

Processed and tokenized answers


,docid,doc,doc_processed,tokenized_ans,ans_len
0,3,I'm not saying I don't like the idea of on-the...,im not saying i dont like the idea of on the j...,"[im, not, saying, i, dont, like, the, idea, of...",76
1,31,So nothing preventing false ratings besides ad...,so nothing preventing false ratings besides ad...,"[so, nothing, preventing, false, ratings, besi...",78
2,56,You can never use a health FSA for individual ...,you can never use a health fsa for individual ...,"[you, can, never, use, a, health, fsa, for, in...",74
3,59,Samsung created the LCD and other flat screen ...,samsung created the lcd and other flat screen ...,"[samsung, created, the, lcd, and, other, flat,...",54
4,63,Here are the SEC requirements: The federal sec...,here are the sec requirements the federal sec...,"[here, are, the, sec, requirements, the, feder...",222


In [ ]:
avg_ans_count = processed_answers['ans_len'].mean()
avg_q_count = processed_questions['q_len'].mean()

print("Average answer length: {}".format(round(avg_ans_count)))
print("Average question length: {}".format(round(avg_q_count)))

Average answer length: 136
Average question length: 11


In [ ]:
print("Total answers: {}".format(len(processed_answers)))
print("Number of answers with length greater than 512: {}".format(len(processed_answers[processed_answers['ans_len'] > 512])))

Total answers: 57600
Number of answers with length greater than 512: 1233


# **Create Vocabulary**

In [ ]:
word2index, word2count = create_vocab(processed_answers, processed_questions)

print("Vocab size: {}".format(len(word2index)))
print("Top {} common words: {}".format(35, Counter(word2count).most_common(35)))

Vocab size: 85034
Top 35 common words: [('the', 371203), ('to', 233559), ('a', 201620), ('you', 166702), ('and', 163066), ('of', 157574), ('is', 129894), ('in', 120019), ('that', 111416), ('for', 89366), ('it', 83822), ('i', 74100), ('your', 68153), ('are', 67255), ('if', 60689), ('be', 59266), ('on', 58382), ('have', 55754), ('as', 50088), ('this', 49868), ('not', 49227), ('or', 46080), ('with', 45894), ('they', 44485), ('but', 41690), ('can', 38863), ('will', 36865), ('at', 35548), ('an', 31392), ('money', 31003), ('so', 29980), ('$', 29096), ('would', 28750), ('from', 28582), ('more', 27378)]


In [ ]:
qid_to_text, docid_to_text = id_to_text(collection, queries)
qid_to_tokenized_text, docid_to_tokenized_text = id_to_tokenized_text(processed_answers, processed_questions)

In [ ]:
# Save objects to pickle
!cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/word2index.pickle", word2index)
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/word2count.pickle", word2count)


# id map to raw text
!cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/id_to_text
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/qid_to_text.pickle", qid_to_text)
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/docid_to_text.pickle", docid_to_text)

# id map to tokenized text
!cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/qid_to_tokenized_text.pickle", qid_to_tokenized_text)
save_pickle("/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/qa_lstm_tokenizer/docid_to_tokenized_text.pickle", docid_to_tokenized_text)

In [ ]:
#!pip install pyserini
!pip install faiss

!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (791 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155639 files and directories currently install

In [ ]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')
hits = searcher.search('what is a lobster roll?')

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [00:31, 70.2MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...
SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.5f}')

 1 7157707 11.00830
 2 6034357 10.94310
 3 5837606 10.81740
 4 7157715 10.59820
 5 6034350 10.48360
 6 2900045 10.31190
 7 7157713 10.12300
 8 1584344 10.05290
 9 533614  9.96350
10 6234461 9.92200


In [ ]:
import pandas as pd
query_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_final.tsv"
query_df = pd.read_csv(query_path, sep="\t")
queries = query_df[['qid', 'question']]

In [ ]:
fiqa_index = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"
searcher = SimpleSearcher(fiqa_index)
hits = None
for i, row in queries.iterrows():
  if i == 5:
    hits = searcher.search(row['question'], k=10)
    break

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.5f}')

 1 480717  6.67740
 2 204747  6.12630
 3 115274  6.06850
 4 522798  5.85410
 5 266163  5.69850
 6 542022  5.59240
 7 274400  5.52160
 8 528206  5.46960
 9 75073   5.26760
10 555430  5.24150


In [ ]:
hits = searcher.search(query, k=cands_size)

Traceback (most recent call last):
  File "./src/generate_data.py", line 15, in <module>
    from pyserini.search import pysearch
  File "/usr/local/lib/python3.7/dist-packages/pyserini/search/__init__.py", line 19, in <module>
    from .lucene import JLuceneSearcherResult, LuceneSimilarities, LuceneFusionSearcher, LuceneSearcher
  File "/usr/local/lib/python3.7/dist-packages/pyserini/search/lucene/__init__.py", line 18, in <module>
    from ._impact_searcher import JImpactSearcherResult, LuceneImpactSearcher
  File "/usr/local/lib/python3.7/dist-packages/pyserini/search/lucene/_impact_searcher.py", line 28, in <module>
    from pyserini.encode import QueryEncoder, TokFreqQueryEncoder, UniCoilQueryEncoder, \
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/__init__.py", line 17, in <module>
    from ._base import DocumentEncoder, QueryEncoder, JsonlCollectionIterator,\
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/_base.py", line 19, in <module>
    impor